In [1]:
#importation des bibliothéque 
import statsmodels.formula.api as s
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime

from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import minmax_scaling
#importation base de donnée
# df = pd.read_csv('/content/drive/MyDrive/stageASM/dataset.csv',header=None)



In [2]:
df = pd.read_csv(r'C:\Users\rabi3\OneDrive\Bureau\freelance\DataSets\Fixing-ML-models_freelancing_Datasets/dataset.csv')

# for testing
dff = df

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ART_Designation  1048575 non-null  object 
 1   price            1048575 non-null  float64
 2   ddm              1048575 non-null  object 
 3   qte              1048575 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 32.0+ MB


In [4]:
le = LabelEncoder()
df["ART_code"] = le.fit_transform(df["ART_Designation"])

In [5]:
# mix-max scale the data between 0 and 1
# minmax_scaling is a function loaded from library
df['ART_scaled']= minmax_scaling(df["ART_code"] , columns=[0])

c:\Users\rabi3\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\preprocessing\scaling.py:40: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  ary_new = ary_new[:, np.newaxis]


In [6]:
# number Of Prodect
df['ART_Designation'].value_counts().count()

14307

In [7]:
# delete product with negative quantity
def delete_Qte_Negative (df):
    return df[df['qte']>0]

In [8]:
# delete product with negative price
def delete_Price_Negative (df):
    return df[df['price']>0]

In [9]:
df = delete_Qte_Negative (df)
df = delete_Price_Negative (df)

In [10]:
# nfaskhou les produits eli ma tbe3oush barsha
# mathalan produit tbe3 mara wa7da fi dataset fiha malyoun vente lazem yetfasakh khater yetsama outliers w ynjaem 
# y8altelna el model 
#  we delete products which had been sold less than 50 times in the dataset
def delete_Outliers(df):
    return df.groupby('ART_Designation').filter(lambda x : len(x)>50)

In [44]:
#  we delete << foreach product >> products which had been sold less than 10 times "grouped by quantity"  in the dataset
def delete_product_Outliers(df):
    return df.groupby(['ART_Designation','qte']).filter(lambda x : len(x)>15)

In [45]:
df = delete_Outliers(df)
df = delete_product_Outliers(df)

In [ ]:
df['day'] = pd.DatetimeIndex(df['ddm']).day
df['month'] = pd.DatetimeIndex(df['ddm']).month
df['day_of_week'] = pd.DatetimeIndex(df['ddm']).weekday

In [47]:
df = df.reset_index()

In [48]:
df_perDay = df[['ART_code','day','month','qte']].copy()
df_perWeek = df[['ART_code','day_of_week','month','qte']].copy()

In [49]:
# day from 1 to 31
df[df['ART_Designation']=="EAU M. SAFIA 1.5L /QT6"]
print(df_perDay, df_perWeek)

        ART_code  day  month  qte
0           5381   31     12    2
1           2480   31     12    1
2          13256   31     12    2
3          11687   31     12    2
4           3407   31     12    6
...          ...  ...    ...  ...
833044      2055   20     10    1
833045      7602   20     10    1
833046      4580   20     10    1
833047      7502   20     10    1
833048      1438   20     10    2

[833049 rows x 4 columns]         ART_code  day_of_week  month  qte
0           5381            4     12    2
1           2480            4     12    1
2          13256            4     12    2
3          11687            4     12    2
4           3407            4     12    6
...          ...          ...    ...  ...
833044      2055            2     10    1
833045      7602            2     10    1
833046      4580            2     10    1
833047      7502            2     10    1
833048      1438            2     10    2

[833049 rows x 4 columns]


In [50]:
x=df_perDay[['ART_code', 'day', 'month']]
x2=df_perWeek[['ART_code', 'day_of_week', 'month']]
y=df_perDay['qte']

In [51]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

In [52]:
reg=LinearRegression()
reg.fit(x_train,y_train)
print ( reg. intercept_ ) 
print ( reg. coef_ )

2.7364374468945347
[-3.47697823e-05 -6.21985294e-03  8.05132716e-04]


In [53]:
#Here x.values will have only values without headers
y_pred=reg.predict(x_test)

In [54]:
reg_df=pd.DataFrame()
reg_df['originale']=y_test
reg_df['Predictions']=y_pred
reg_df

,originale,Predictions
592636,1,2.243046
619761,1,2.338341
43473,2,2.366696
798801,2,2.445044
572310,1,2.710659
...,...,...
10921,1,2.079496
188362,3,2.551533
555920,1,2.406737
329785,1,2.205700


In [55]:
x2_train,x2_test,y_train,y_test=train_test_split(x2,y,test_size=0.1)

In [56]:
reg=LinearRegression()
reg.fit(x2_train,y_train)
print ( reg. intercept_ ) 
print ( reg. coef_ )

2.6366912035577035
[-3.50264373e-05 -4.85527251e-04  1.20665926e-03]


In [57]:
#Here x.values will have only values without headers
y_pred=reg.predict(x2_test)

In [58]:
reg_df=pd.DataFrame()
reg_df['originale']=y_test
reg_df['Predictions']=y_pred
reg_df

,originale,Predictions
248469,2,2.295281
322315,1,2.525618
140215,2,2.302389
199723,1,2.340244
428295,12,2.164325
...,...,...
61173,1,2.168148
619700,4,2.358035
461370,2,2.395453
190193,1,2.282615
